In [1]:
# Imports
import tensorflow as tf
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import os
import numpy
from skimage import color, io
from skimage.io import imsave
import scipy.io

In [2]:
def get_model_data():
    data = numpy.load("../vgg/vgg_model.npy")
    return data[()]

def weight(width, height, input_channels, output_channels, variable_name):
    # [width, height, input channel, output channel]
    return tf.get_variable(variable_name, initializer=tf.truncated_normal([width, height, input_channels, output_channels], stddev=0.02))

def bias(output_channels, variable_name):
    return tf.get_variable(variable_name, initializer=tf.constant(0.0, shape=[output_channels]))

def maxPool(X):
    return tf.nn.max_pool(X, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

def conv2dTranspose(X, W, B, output_shape, stride=2):
    # Lesa þetta: http://www.matthewzeiler.com/wp-content/uploads/2017/07/cvpr2010.pdf
    conv2d = tf.nn.conv2d_transpose(X, W, output_shape, strides=[1, stride, stride, 1], padding="SAME")
    return tf.nn.bias_add(conv2d, B)

In [3]:
def generateVGGLayers(input_layer):
    vgg_model = get_model_data()
    vgg_network = {}

    def vggVariable(values, name):
        # Populate tensor with values
        return tf.get_variable(
            name=name, 
            initializer=tf.constant_initializer(values, dtype=tf.float32), 
            shape=values.shape
        )

    def mat2tf(kernels):
        # matconvnet: [width, height, in_channels, out_channels]
        # tensorflow: [height, width, in_channels, out_channels]
        return numpy.transpose(kernels, (1, 0, 2, 3))

    def vggConv(name, X):
        weight = mat2tf(vgg_model[name]['kernels'])
        bias = vgg_model[name]['bias'].reshape(-1) # flatten

        weight = vggVariable(weight, "vgg/" + name + "/weight")
        bias = vggVariable(bias, "vgg/" + name + "/bias")

        X = tf.nn.conv2d(X, weight, strides=[1, 1, 1, 1], padding="SAME")
        return tf.nn.bias_add(X, bias)

    def vggRelu(name, X):
        return tf.nn.relu(X, name="vgg/" + name + "/relu")
    
    def vggPool(X):
        return tf.nn.avg_pool(X, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

    # Hidden layer 1
    vgg_network['conv1_2'] = vggConv('conv1_2', input_layer)
    vgg_network['relu1_2'] = vggRelu('relu1_2', vgg_network['conv1_2'])
    vgg_network['pool1'] = vggPool(vgg_network['relu1_2'])

    # Hidden layer 2
    vgg_network['conv2_1'] = vggConv('conv2_1', vgg_network['pool1'])
    vgg_network['relu2_1'] = vggRelu('relu2_1', vgg_network['conv2_1'])
    vgg_network['conv2_2'] = vggConv('conv2_2', vgg_network['relu2_1'])
    vgg_network['relu2_2'] = vggRelu('relu2_2', vgg_network['conv2_2'])
    vgg_network['pool2'] = vggPool(vgg_network['relu2_2'])

    # Hidden layer 3
    vgg_network['conv3_1'] = vggConv('conv3_1', vgg_network['pool2'])
    vgg_network['relu3_1'] = vggRelu('relu3_1', vgg_network['conv3_1'])
    vgg_network['conv3_2'] = vggConv('conv3_2', vgg_network['relu3_1'])
    vgg_network['relu3_2'] = vggRelu('relu3_2', vgg_network['conv3_2'])
    vgg_network['conv3_3'] = vggConv('conv3_3', vgg_network['relu3_2'])
    vgg_network['relu3_3'] = vggRelu('relu3_3', vgg_network['conv3_3'])
    vgg_network['conv3_4'] = vggConv('conv3_4', vgg_network['relu3_3'])
    vgg_network['relu3_4'] = vggRelu('relu3_4', vgg_network['conv3_4'])
    vgg_network['pool3'] = vggPool(vgg_network['relu3_4'])

    # Hidden layer 4
    vgg_network['conv4_1'] = vggConv('conv4_1', vgg_network['pool3'])
    vgg_network['relu4_1'] = vggRelu('relu4_1', vgg_network['conv4_1'])
    vgg_network['conv4_2'] = vggConv('conv4_2', vgg_network['relu4_1'])
    vgg_network['relu4_2'] = vggRelu('relu4_2', vgg_network['conv4_2'])
    vgg_network['conv4_3'] = vggConv('conv4_3', vgg_network['relu4_2'])
    vgg_network['relu4_3'] = vggRelu('relu4_3', vgg_network['conv4_3'])
    vgg_network['conv4_4'] = vggConv('conv4_4', vgg_network['relu4_3'])
    vgg_network['relu4_4'] = vggRelu('relu4_4', vgg_network['conv4_4'])
    vgg_network['pool4'] = vggPool(vgg_network['relu4_4'])

    # Hidden layer 5
    vgg_network['conv5_1'] = vggConv('conv5_1', vgg_network['pool4'])
    vgg_network['relu5_1'] = vggRelu('relu5_1', vgg_network['conv5_1'])
    vgg_network['conv5_2'] = vggConv('conv5_2', vgg_network['relu5_1'])
    vgg_network['relu5_2'] = vggRelu('relu5_2', vgg_network['conv5_2'])
    vgg_network['conv5_3'] = vggConv('conv5_3', vgg_network['relu5_2'])
    vgg_network['relu5_3'] = vggRelu('relu5_3', vgg_network['conv5_3'])
    vgg_network['conv5_4'] = vggConv('conv5_4', vgg_network['relu5_3'])
    vgg_network['relu5_4'] = vggRelu('relu5_4', vgg_network['conv5_4'])

    return vgg_network

def createNetwork(L):
    # Input layer
    W = weight(width=3, height=3, input_channels=1, output_channels=64, variable_name="input_layer/weight")
    B = bias(output_channels=64, variable_name="input_layer/bias")
    input_layer = tf.nn.bias_add(tf.nn.conv2d(L, W, strides=[1, 1, 1, 1], padding="SAME"), B)
    input_layer = tf.nn.relu(input_layer, name="input_later/relu")

    # VGG layers
    vgg_network = generateVGGLayers(input_layer)
    vgg_pool5 = maxPool(vgg_network["relu5_3"])

    # Hidden layer 1 (scale up(pool5) + pool4)
    vgg_pool4 = vgg_network["pool4"]
    W1 = weight(4, 4, vgg_pool4.shape[3].value, vgg_pool5.shape[3].value, "hidden_layer/1/weight")
    B1 = bias(vgg_pool4.shape[3].value, "hidden_layer/1/bias")
    conv_trans1 = conv2dTranspose(vgg_pool5, W1, B1, output_shape=tf.shape(vgg_pool4))
    hypercolumns = tf.add(conv_trans1, vgg_pool4, name="hidden_layer/1/fuse")

    # Hidden layer 2 (scale up (scale up(pool5) + pool4) + pool3)
    vgg_pool3 = vgg_network["pool3"]
    W2 = weight(4, 4, vgg_pool3.shape[3].value, vgg_pool4.shape[3].value,"hidden_layer/2/weight")
    B2 = bias(vgg_pool3.shape[3].value, "hidden_layer/2/bias")
    conv_trans2 = conv2dTranspose(hypercolumns, W2, B2, output_shape=tf.shape(vgg_pool3))
    hypercolumns = tf.add(conv_trans2, vgg_pool3, name="hidden_layer/2/fuse")
    
    # Output layer (scale up (scale up (scale up(pool5) + pool4) + pool3) to picture size)
    input_shape = tf.shape(L)
    output_shape = tf.stack([input_shape[0], input_shape[1], input_shape[2], 2])
    W3 = weight(16, 16, 2, vgg_pool3.shape[3].value, "output_layer/weight")
    B3 = bias(2, "output_layer/bias")
    AB = conv2dTranspose(hypercolumns, W3, B3, output_shape=output_shape, stride=8)
    
    # Output LAB values
    return tf.concat([L, AB], 3, name="colorized_image") # [?, pic_width, pic_height, 3]

In [4]:
def convertToLab(image):
    lab = color.rgb2lab(image)
    X_batch = lab[:,:,0]
    Y_batch = lab[:,:,1:]
    return X_batch.reshape(X_batch.shape+(1,)), lab

def parseImage(filename):
    image = tf.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_images(image, [400, 400], tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image

In [5]:
file_test_paths = ['woman.jpg']

dataset = tf.data.Dataset.from_tensor_slices(file_test_paths)
dataset = dataset.map(parseImage)
dataset = dataset.map(lambda image: 
    tuple(tf.py_func(
        convertToLab, [image], [tf.double, tf.double]
    ))
)
dataset = dataset.batch(1)

iterator = dataset.make_one_shot_iterator()

In [6]:
X = tf.placeholder(tf.float32, shape=[None, None, None, 1], name='L_image')
Y_ = tf.placeholder(tf.float32, shape=[None, None, None, 3], name="LAB_image")

Y = createNetwork(X)

In [7]:
IMAGE_SIZE = 400
# Define the loss function 
#loss = tf.reduce_mean(2 * tf.nn.l2_loss(Y - Y_)) / (IMAGE_SIZE * IMAGE_SIZE * 100 * 100)
#tf.summary.scalar("loss", loss)
loss = tf.reduce_mean(tf.squared_difference(Y, Y_), 1)
loss_mean = tf.reduce_mean(loss)


# Define an optimizer
optimizer = tf.train.AdamOptimizer(0.0001, beta1=0.9).minimize(loss)

In [8]:
init = tf.global_variables_initializer()
next_element = iterator.get_next()
saver = tf.train.Saver()

with tf.Session() as sess:
  
    # initialize the variables
    sess.run(init)

    # initialize the queue threads to start to shovel data
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    print "from the train set:"
    # images, labels = iterator.get_next()
    step = 0
    while True:
        
        try:
            elem = sess.run(next_element)
            print "Step:", step
            for i in range(10):
                print "Round:", i
                _, luss = sess.run([optimizer, loss_mean], feed_dict={
                    X: elem[0], Y_: elem[1]
                })
                print "Loss:", luss
            step += 1
        except tf.errors.OutOfRangeError:
            saver.save(sess, './model/' + 'model.ckpt', global_step=step+1)
            print("End of training dataset.")
            break
            
    # stop our queue threads and properly close the session
    coord.request_stop()
    coord.join(threads)
    sess.close()

from the train set:
Step: 0
Round: 0
Loss: 138.879
Round: 1
Loss: 110.635
Round: 2
Loss: 99.709
Round: 3
Loss: 94.3805
Round: 4
Loss: 88.827
Round: 5
Loss: 80.8492
Round: 6
Loss: 71.7452
Round: 7
Loss: 72.8402
Round: 8
Loss: 61.305
Round: 9
Loss: 58.8664
End of training dataset.
